#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

l_d =df_ks.to_dict(orient='records')

### Importer les données

In [6]:
collection.insert_many(l_d)

## Question 1  

In [7]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': datetime.datetime(2016, 5, 10, 15, 53, 56),
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': datetime.datetime(2014, 10, 23, 20, 6, 37),
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2017-08-11',
  'goal': '35000.0'

## Question 2

In [21]:
cur =collection.find({"state":"successful"})
len(list(cur))

53040

## Question 3

In [22]:
cur = collection.aggregate([{"$group" : {"_id":"$category","Project" : {"$sum" :1}}}])
list(cur)

[{'_id': 'Calendars', 'Project': 112},
 {'_id': 'Webcomics', 'Project': 259},
 {'_id': 'Interactive Design', 'Project': 146},
 {'_id': 'Textiles', 'Project': 105},
 {'_id': 'Radio & Podcasts', 'Project': 349},
 {'_id': 'Animals', 'Project': 97},
 {'_id': 'Festivals', 'Project': 313},
 {'_id': 'Literary Journals', 'Project': 118},
 {'_id': 'Chiptune', 'Project': 10},
 {'_id': 'Kids', 'Project': 109},
 {'_id': 'Documentary', 'Project': 6498},
 {'_id': 'Letterpress', 'Project': 24},
 {'_id': 'Action', 'Project': 282},
 {'_id': 'Wearables', 'Project': 508},
 {'_id': 'Periodicals', 'Project': 517},
 {'_id': 'Embroidery', 'Project': 49},
 {'_id': 'Technology', 'Project': 2690},
 {'_id': 'Comics', 'Project': 1931},
 {'_id': 'Robots', 'Project': 237},
 {'_id': 'Community Gardens', 'Project': 115},
 {'_id': "Farmer's Markets", 'Project': 175},
 {'_id': 'Video Games', 'Project': 4799},
 {'_id': 'Performance Art', 'Project': 855},
 {'_id': 'Comedy', 'Project': 923},
 {'_id': 'Dance', 'Project': 9

## Question 4

In [33]:
cur =collection.find({"$and":[{"launched":{"$gt":"datetime.datetime(2016, 0, 0, 0, 0, 0)"}},{"country":"FR"})
list(cur)

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-33-b0f22d12ef59>, line 1)

## Question 5

## Question 6 